In [15]:
import torchvision
import torch
# root_dir='D:\code_file\dplearning_xiaotutui\data'
# dataset=torchvision.datasets.ImageNet(root_dir, download=True,split='train',transform=torchvision.transforms.ToTensor())
# 数据集太大了放弃
# vgg_par_non.load_state_dict(torch.load('vgg16.pth'))

In [39]:

###模型随机化参数
vgg16=torchvision.models.vgg16()
###参数路径
par_path=r'D:\arc_file\dplearning_relate\model\vgg16-397923af.pth'
##模型加载参数
vgg16.load_state_dict(torch.load(par_path))

<All keys matched successfully>

这是vgg16层 表明16层有参数
设计思路
小卷积核优势：

多个3x3卷积叠加 ≈ 1个5x5或7x7卷积的感受野，但参数更少、非线性更多。
两个3x3=5x5 因为3+2=5
例如：3层3x3卷积的参数量为 3*(3^2*C^2) = 27C²，而1层7x7卷积为 49C²。
通道数翻倍：

每次池化后通道数翻倍（64→128→256→512），平衡特征图尺寸减小带来的信息损失。
全连接层压缩：

通过Dropout和ReLU防止过拟合，逐步将特征映射到类别空间。
VGG(
###这里是特征提取器
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (18): ReLU(inplace=True)
    (19): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (20): ReLU(inplace=True)
    (21): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (22): ReLU(inplace=True)
    (23): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (24): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (25): ReLU(inplace=True)
    (26): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (27): ReLU(inplace=True)
    (28): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (29): ReLU(inplace=True)
    (30): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
###这里是自适应平均池化 将不同尺寸的输入统一调整为 7x7（例如原图不是224x224时也能处理）
  (avgpool): AdaptiveAvgPool2d(output_size=(7, 7))
 ###这里是分类器（classifier）
  (classifier): Sequential(
    (0): Linear(in_features=25088, out_features=4096, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=4096, out_features=4096, bias=True)
    (4): ReLU(inplace=True)
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=4096, out_features=1000, bias=True)
  )
)

In [40]:
from torch import nn

###如何利用现有网络去修改结构,当前vgg网络输出1000，我需要预测一个10分类的任务因此需要调整网络
# vgg16.add_module('add_linear',nn.Sequential(nn.Linear(1000,10)))
###如果要加到现有的模型内
# vgg16.classifier.add_module('add_linear',nn.Linear(1000,10))

###修改现有模型
vgg16.classifier[6]=nn.Linear(4096,10)